In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import glob

In [ ]:
ds = xr.open_dataset('/resstore/b0243/Data/earfw/FX2100f19f19mg16NaFeMgiontransport/hist/FX2100f19f19mg16NaFeMgiontransport.cam.h2.2100-01-01-00000.nc')
ds

In [ ]:
def plot_daily_average_zonal_wind(dataset, level_index=0):
    u_wind_avg = dataset.U.mean(dim='time')

    u_wind = u_wind_avg[level_index, :, :]

    fig = plt.figure(figsize=(15, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())

    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.gridlines()

    lons = dataset.lon.values
    lats = dataset.lat.values
    lon2d, lat2d = np.meshgrid(lons, lats)

    levels = np.linspace(u_wind.min(), u_wind.max(), 20)
    cf = ax.contourf(lon2d, lat2d, u_wind, 
                     transform=ccrs.PlateCarree(),
                     levels=levels,
                     cmap='RdBu_r',
                     extend='both')

    plt.colorbar(cf, ax=ax, orientation='horizontal', 
                label='Daily Average Zonal Wind Speed (m/s)',
                pad=0.05)
    
    pressure_level = dataset.lev[level_index].values
    date_str = str(dataset.time[0].values).split(' ')[0] 
    plt.title(f'Daily Average Zonal Wind at Level {pressure_level:.2e} hPa\n{date_str}')
    
    return fig

fig = plot_daily_average_zonal_wind(ds, level_index=50)
plt.show()

In [ ]:
def plot_monthly_wind(base_path, month, level_index=0):

    file_pattern = base_path.format(month=month, day='*')

    print(f"Looking for files matching pattern: {file_pattern}")
    files = glob.glob(file_pattern)
    print(f"Found {len(files)} files")

    ds = xr.open_mfdataset(files, combine='nested', concat_dim='time')

    u_wind_avg = ds.U.mean(dim='time')
    u_wind = u_wind_avg[level_index, :, :]

    fig = plt.figure(figsize=(15, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())

    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.gridlines()

    lons = ds.lon.values
    lats = ds.lat.values
    lon2d, lat2d = np.meshgrid(lons, lats)

    levels = np.linspace(u_wind.min(), u_wind.max(), 20)
    cf = ax.contourf(lon2d, lat2d, u_wind, 
                     transform=ccrs.PlateCarree(),
                     levels=levels,
                     cmap='RdBu_r',
                     extend='both')

    plt.colorbar(cf, ax=ax, orientation='horizontal', 
                label='Monthly Average Zonal Wind Speed (m/s)',
                pad=0.05)

    pressure_level = ds.lev[level_index].values
    month_name = {
        '01': 'January', '02': 'February', '03': 'March',
        '04': 'April', '05': 'May', '06': 'June',
        '07': 'July', '08': 'August', '09': 'September',
        '10': 'October', '11': 'November', '12': 'December'
    }[month]
    
    plt.title(f'Monthly Average Zonal Wind at Level {pressure_level:.2e} hPa\n{month_name} 2100')
    
    ds.close()
    
    return fig

In [ ]:
base_path = "/resstore/b0243/Data/earfw/FX2100f19f19mg16NaFeMgiontransport/hist/FX2100f19f19mg16NaFeMgiontransport.cam.h2.2100-{month}-{day}-*.nc"
fig = plot_monthly_wind(base_path, month='01', level_index=50)
plt.show()

In [2]:
def plot_all_monthly_winds(base_path, level_index=0):

    fig = plt.figure(figsize=(20, 15))

    month_names = {
        '01': 'January', '02': 'February', '03': 'March',
        '04': 'April', '05': 'May', '06': 'June',
        '07': 'July', '08': 'August', '09': 'September',
        '10': 'October', '11': 'November', '12': 'December'
    }
    
    all_wind_data = []
    
    for month in [f"{i:02d}" for i in range(1, 13)]: # Creates the array [01, 02, 03, ...]
        file_pattern = base_path.format(month=month, day='*')
        files = glob.glob(file_pattern)
            
        ds = xr.open_mfdataset(files, combine='nested', concat_dim='time')
        u_wind_avg = ds.U.mean(dim='time')
        all_wind_data.append(u_wind_avg[level_index, :, :])
        ds.close()

    vmin = min(data.min() for data in all_wind_data)
    vmax = max(data.max() for data in all_wind_data)
    levels = np.linspace(vmin, vmax, 20) # Consistant colour bar
    
    for idx, (month, wind_data) in enumerate(zip([f"{i:02d}" for i in range(1, 13)], all_wind_data)):
        ax = fig.add_subplot(3, 4, idx + 1, projection=ccrs.PlateCarree())
        
        ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
        ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5)
        ax.gridlines(linewidth=0.5, alpha=0.5)

        if idx == 0: 
            lons = ds.lon.values
            lats = ds.lat.values
            pressure_level = ds.lev[level_index].values
        
        lon2d, lat2d = np.meshgrid(lons, lats)

        cf = ax.contourf(lon2d, lat2d, wind_data,
                        transform=ccrs.PlateCarree(),
                        levels=levels,
                        cmap='RdBu_r',
                        extend='both')

        ax.set_title(f'{month_names[month]}', fontsize=10)

        # z3_var = ds.variables['Z3']
        # h_geopotential = z3_var[0, level_index, :, :]
        # R_e = 6371000.0
        # z_geometric = (R_e * h_geopotential) / (R_e - h_geopotential)
        # mean_altitude_km = np.mean(z_geometric) / 1000.0

    fig.suptitle(f'Monthly Average Zonal Wind at Level {pressure_level:.2e} hPa - 2100',
                 fontsize=14, y=0.95)
    
    cbar_ax = fig.add_axes([0.1, 0.05, 0.8, 0.02])
    cbar = fig.colorbar(cf, cax=cbar_ax, orientation='horizontal',
                       label='Zonal Wind Speed (m/s)')
    
    plt.subplots_adjust(left=0.05, right=0.95, top=0.9, bottom=0.1, hspace=0.3)
    
    return fig

In [ ]:
base_path = "/resstore/b0243/Data/earfw/FX2100f19f19mg16NaFeMgiontransport/hist/FX2100f19f19mg16NaFeMgiontransport.cam.h2.2100-{month}-{day}-*.nc"
fig = plot_all_monthly_winds(base_path, level_index=54)
plt.show()